In [1]:
from dotenv import load_dotenv
import inspect
import os
import pandas as pd

from utils import init_flipside, execute_query

from queries import (
    bridge,
    dex,
    farcaster,
    lending,
    marketplace,
    nft,
    rwa,
    staking
)

In [2]:
load_dotenv()
FLIPSIDE_API_KEY = os.getenv('FLIPSIDE_API_KEY')
flipside = init_flipside(FLIPSIDE_API_KEY)
apps = pd.read_csv('data/applications_reviewed.csv', index_col=0)

In [3]:
METRIC_MODULES = {
    'Art NFTs': {
        "module": nft,
        "params": ["address"]
    },
    'Other Media NFTs': {
        "module": nft,
        "params": ["address"]
    },
    'Bridges': {
        "module": bridge,
        "params": ["chain", "address"]
    },
    'DEX': {
        "module": dex,
        "params": ["chain", "address"]
    },
    'Channels': {
        "module": farcaster,
        "params": ["warpcast_url"]
    },
    'Frames': {
        "module": None, # needs to be updated
        "params": ["warpcast_url"]
    },
    'Lending': {
        "module": lending,
        "params": ["chain", "address"]
    },
    'Art Marketplace': {
        "module": marketplace,
        "params": ["chain", "address"]
    },
    'Other Media Marketplace': {
        "module": marketplace,
        "params": ["chain", "address"]
    },
    'Real World Assets': {
        "module": rwa,
        "params": ["chain", "address"]
    },
    'Staking': {
        "module": staking,
        "params": ["chain", "address"]
    },
    'Community & Curation': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Gaming': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Airdrop': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Meme Community': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Payments': {
        "module": None,
        "params": ["chain", "address"]
    },    
    'Donations': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Identity': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Betting & Prediction Markets': {
        "module": None,
        "params": ["chain", "address"]
    },
}

In [4]:
def query_project_metrics(project_data):    
    uid = project_data['id']
    category = project_data['category']
    module_name = METRIC_MODULES.get(category).get('module')
    if not module_name:
        return
    
    metric_modules = inspect.getmembers(module_name, inspect.isfunction)
    expected_params = METRIC_MODULES.get(category).get('params', [])
    params = [project_data.get(p) for p in expected_params]
    if len(params) != len(expected_params):
        print("Unable to find params:", expected_params)
        return
    
    metrics = []
    project_info = {'id': uid, 'category': category}
    project_info.update(dict(zip(expected_params, params)))
    for (query_name, query_fn) in metric_modules:
        sql = query_fn(*params)
        value = execute_query(flipside, sql, project_info)
        metric_info = project_info.copy()
        metric_info.update({'query_name': query_name, 'value': value})
        metrics.append(metric_info)
    return metrics

In [5]:
apps.iloc[0]

id                        0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406...
recipient                        0xE4EE538019673501F4B75de5aF5CC073Ec0A1487
time                                                             1724860607
name                                                                     0x
profile_name                                                     ewokafloka
profile_url                                 https://warpcast.com/ewokafloka
metadata_name                                                            0x
metadata_website                                            https://0x.org/
project_type                                                            App
category                                                                DEX
address_type                                                       contract
address                          0xdef1c0ded9bec7f1a1670819833240f027b25eff
chain_id                                                             8453.0
chain       

In [6]:
results = query_project_metrics(apps.iloc[0])
results

2024-09-16 10:09:15,248 - INFO - Query Attempt: id: 0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab, category: DEX, chain: Base, address: 0xdef1c0ded9bec7f1a1670819833240f027b25eff, Status: success, Execution Time: 83.03 seconds, Value: 1646413
2024-09-16 10:09:27,208 - INFO - Query Attempt: id: 0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab, category: DEX, chain: Base, address: 0xdef1c0ded9bec7f1a1670819833240f027b25eff, Status: success, Execution Time: 11.96 seconds, Value: 0
2024-09-16 10:09:35,124 - ERROR - Query execution failed: QUERY_RUN_EXECUTION_ERROR: an error has occured while executing your query. errorName=OperationFailedError, errorMessage=SQL compilation error:
syntax error line 13 at position 39 unexpected '60'.
syntax error line 13 at position 46 unexpected 'PRECEDING'., errorData={'code': '001003', 'data': {'age': 0, 'pos': -1, 'line': -1, 'type': 'COMPILATION', 'queryId': '01b71211-0410-eae1-3d4f-83026d7485b3', 'sqlState': '4200

[{'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_ntrxns',
  'value': 1646413},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_num_tokens',
  'value': 0},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_nusers',
  'value': None},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_sumtrxns',
  'value': 22188.398141304}]